<a href="https://colab.research.google.com/github/spydann/Opay-Play-Store-Sentiment-Analysis/blob/main/Opay_Sentiment_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-play-scraper

In [ ]:
#from google_play_scraper import reviews, Sort
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from wordcloud import WordCloud
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
app_id = 'team.opay.pay'

result, continuation_token = reviews(
    app_id,
    lang='en',  # Language
    country='ng',  # Country
    sort=Sort.NEWEST,  # Sorting order
    count=500000  # Number of reviews to fetch
)


In [ ]:
# Print the first 10 reviews
for review in result[:10]:
    print(f"User: {review['userName']}")
    print(f"Rating: {review['score']}")
    print(f"Review: {review['content']}\n")

In [ ]:
#from google.colab import files
# create dataframe
reviews_df  = pd.DataFrame(result)

# save file to csv
reviews_df.to_csv('/content/drive/MyDrive/reviews.csv', index=False)

# download the csv file
files.download('/content/drive/MyDrive/reviews.csv')

In [ ]:
# read csv file
reviews_df = pd.read_csv('/content/drive/MyDrive/reviews.csv')

reviews_df.tail()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
257747,2302d5d7-f0e1-459d-8b10-7eec958f00d9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,How do it work to earn..pls urgently..,5,17,1.5.1.75233759,2018-06-18 11:40:07,NaN,NaN,1.5.1.75233759
257748,041215e6-0900-4274-9ed0-39de9e15bbd4,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Cool app,5,17,1.5.0.73642733,2018-06-15 11:28:17,NaN,NaN,1.5.0.73642733
257749,1b576413-0403-4b4e-8f4d-592ec1355230,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Nice,5,16,NaN,2018-06-13 12:06:00,NaN,NaN,NaN
257750,ffe87784-e781-43bd-9d3e-498ab3abe4c8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Good,5,16,NaN,2018-06-11 10:35:52,NaN,NaN,NaN
257751,05d64281-5e76-4dd9-860d-0b7cde8633a3,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Great stuff loading,5,27,1.4.0.71852517,2018-06-04 06:48:25,NaN,NaN,1.4.0.71852517


In [ ]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257752 entries, 0 to 257751
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              257752 non-null  object
 1   userName              257752 non-null  object
 2   userImage             257752 non-null  object
 3   content               257740 non-null  object
 4   score                 257752 non-null  int64 
 5   thumbsUpCount         257752 non-null  int64 
 6   reviewCreatedVersion  205584 non-null  object
 7   at                    257752 non-null  object
 8   replyContent          245409 non-null  object
 9   repliedAt             245409 non-null  object
 10  appVersion            205584 non-null  object
dtypes: int64(2), object(9)
memory usage: 21.6+ MB


In [ ]:
# drop unneccessary columns
reviews_df.drop(['reviewId','userName', 'userImage', 'reviewCreatedVersion','replyContent'], axis=1, inplace=True)

reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion
0,Opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237
2,Opay is really a good app. Easy and good for t...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,NaN
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,NaN
4,Opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237


In [ ]:
# change 'repliedAt' and at columns to datetime object
reviews_df['repliedAt'] = pd.to_datetime(reviews_df['repliedAt'])
reviews_df['at'] = pd.to_datetime(reviews_df['at'])

reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion
0,Opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237
2,Opay is really a good app. Easy and good for t...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,NaN
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,NaN
4,Opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237


In [ ]:
# deal with missing values
reviews_df['appVersion'].fillna('Unknown', inplace=True)

reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion
0,Opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237
2,Opay is really a good app. Easy and good for t...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,Unknown
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,Unknown
4,Opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237


In [ ]:
reviews_df['appVersion'].nunique()

342

##Clean text column

In [ ]:
# clean content column
# Function to remove non-letter and non-digit characters
def preprocess_text(text):
    # Handle non-string values
    if not isinstance(text, str):
        return '' # Or any other suitable handling for non-string values

    emoji_removed = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    lowered = emoji_removed.lower()
    cleaned = re.sub(r'\s+', ' ', lowered)

    return cleaned

# Apply the function to the 'content' column
reviews_df['content'] = reviews_df['content'].apply(preprocess_text)

reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion
0,opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237
2,opay is really a good app easy and good for tr...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,Unknown
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,Unknown
4,opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237


##Perform Sentiment Analysis using Vader

VADER is a powerful tool for sentiment analysis in scenarios where text is informal and contains punctuation, capitalization, and emoticons.

In [ ]:
# Initialize the VADER sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# get polarity score for the first 10
for text in reviews_df['content'].values[:10]:
    sentiment = sia.polarity_scores(text)
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print()

Text: opay is the best
Sentiment: {'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.6369}

Text: very good
Sentiment: {'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'compound': 0.4927}

Text: opay is really a good app easy and good for transfers
Sentiment: {'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.8608}

Text: opay is super easy and fast when it comes to banking thats why i love it
Sentiment: {'neg': 0.0, 'neu': 0.522, 'pos': 0.478, 'compound': 0.9001}

Text: opay is the best ever
Sentiment: {'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.6369}

Text: good
Sentiment: {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4404}

Text: goodluck
Sentiment: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Text: abubakarishaq
Sentiment: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Text: great product and timely
Sentiment: {'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.6249}

Text: good app
Sentiment: {'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'compound':

**Breakdown of Scores:**

*   neg: Proportion of negative sentimet
*   neu: Proportion of neutral sentiment
*   pos: Proportion of positive sentiment
* The compound score is a normalized, weighted composite score ranging from -1 (most negative) to +1 (most positive). It summarizes the overall sentiment of the text.

In [ ]:
# perform sentiment analysis on all text

def get_sentiment_scores(text):
    sentiment = sia.polarity_scores(text)
    return pd.Series([sentiment['neg'], sentiment['neu'], sentiment['pos'], sentiment['compound']])


In [ ]:
# Apply the function to the DataFrame
reviews_df[['Negative', 'Neutral', 'Positive', 'Compound']] = reviews_df['content'].apply(get_sentiment_scores)

# Display the DataFrame
reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion,Negative,Neutral,Positive,Compound
0,opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234,0.0,0.417,0.583,0.6369
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237,0.0,0.238,0.762,0.4927
2,opay is really a good app easy and good for tr...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,Unknown,0.0,0.423,0.577,0.8608
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,Unknown,0.0,0.522,0.478,0.9001
4,opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237,0.0,0.488,0.512,0.6369


In [ ]:
# categorize text based on Compound score

def categorize_sentiment(score):
    if score < -0.05:
        return 'Negative'
    elif score > 0.05:
        return 'Positive'
    else:
        return 'Neutral'

reviews_df['Sentiment'] = reviews_df['Compound'].apply(categorize_sentiment)

reviews_df.head()

,content,score,thumbsUpCount,at,repliedAt,appVersion,Negative,Neutral,Positive,Compound,Sentiment
0,opay is the best,5,0,2024-06-29 17:53:04,2024-06-29 18:22:51,7.9.0.234,0.0,0.417,0.583,0.6369,Positive
1,very good,5,0,2024-06-29 17:51:44,2024-06-29 18:23:32,7.10.1.237,0.0,0.238,0.762,0.4927,Positive
2,opay is really a good app easy and good for tr...,5,0,2024-06-29 17:44:36,2024-06-29 18:24:01,Unknown,0.0,0.423,0.577,0.8608,Positive
3,opay is super easy and fast when it comes to b...,5,0,2024-06-29 17:41:43,2024-06-29 18:24:40,Unknown,0.0,0.522,0.478,0.9001,Positive
4,opay is the best ever,5,0,2024-06-29 17:37:04,2023-06-10 08:07:46,7.10.1.237,0.0,0.488,0.512,0.6369,Positive


In [ ]:
# save to csv and download
from google.colab import files

reviews_df.to_csv('/content/drive/MyDrive/reviews_sentiment.csv', index=False)
files.download('/content/drive/MyDrive/reviews_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Text Preprocessing and Word Frequency Analysis

In [ ]:
# Initialize stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# Function to preprocess text
def text_preprocessor(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove stop words and non-alphabetic tokens
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

# Apply preprocessing to the DataFrame
reviews_df['tokens'] = reviews_df['content'].apply(text_preprocessor)

In [ ]:
# Functions to count word frequencies based on sentiment
def count_words_by_sentiment(df, sentiment):
    # Filter comments by sentiment
    filtered_df = df[df['Sentiment'] == sentiment]
    # Flatten list of tokens
    all_tokens = [token for tokens in filtered_df['tokens'] for token in tokens]
    # Count word frequencies
    word_counts = Counter(all_tokens)
    # Convert to DataFrame
    word_freq_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency'])
    word_freq_df['Sentiment'] = sentiment
    return word_freq_df



In [ ]:
# count word frequencies for each sentiment
positive_freq_df = count_words_by_sentiment(reviews_df, 'Positive')
negative_freq_df = count_words_by_sentiment(reviews_df, 'Negative')
neutral_freq_df = count_words_by_sentiment(reviews_df, 'Neutral')

# combine dataframes
word_freq_df = pd.concat([positive_freq_df, negative_freq_df, neutral_freq_df])

In [ ]:
word_freq_df.head(50)

,Word,Frequency,Sentiment
0,opay,28762,Positive
1,best,26015,Positive
2,good,58112,Positive
3,really,7017,Positive
4,app,73638,Positive
5,easy,10625,Positive
6,transfer,5232,Positive
7,super,1130,Positive
8,fast,9921,Positive
9,come,1050,Positive


In [ ]:
# merge df
merged_df = pd.merge(reviews_df, word_freq_df, on='Sentiment', how='left')
merged_df.head()

In [ ]:
# save to csv
from google.colab import files

word_freq_df.to_csv('/content/drive/MyDrive/word_freq.csv', index=False)
files.download('/content/drive/MyDrive/word_freq.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Count word frequencies
word_freq = Counter(all_tokens)

# Get the top 10 most common words
top_10_words = word_freq.most_common(10)

# Display the top 10 words
print(top_10_words)

[('app', 89212), ('good', 60268), ('opay', 38736), ('best', 26392), ('love', 22112), ('nice', 20351), ('money', 15559), ('fast', 14729), ('great', 14531), ('account', 11973)]


In [ ]:
word_freq_df = pd.DataFrame(word_freq.keys(), columns=['Words'])
word_freq_df['Frequency'] = word_freq.values()

word_freq_df.head()

,Words,Frequency
0,opay,38736
1,best,26392
2,good,60268
3,really,8483
4,app,89212


In [ ]:
# save word freq df to csv and download

word_freq_df.to_csv('/content/drive/MyDrive/word_freq.csv', index=False)
files.download('/content/drive/MyDrive/word_freq.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Visualizing Sentiment Analysis and Word Frequency



> Using Power BI





[Power BI Dashboard](https://app.powerbi.com/view?r=eyJrIjoiYzNjZGNiMmUtODM5ZC00ZDE0LTgxYjAtYzQzYTJlZDBkOTIwIiwidCI6IjFiYTIwODY1LWY3ZWYtNGM0Mi1hOTNjLTlhNGFjZGI0YzM2ZSJ9&pageName=ReportSection)